In [3]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [4]:
from sklearn import datasets
from sklearn import metrics as m
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from itertools import combinations
from sklearn import datasets
from sklearn import metrics as m
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.svm import SVC
import xgboost as xgb
from thundersvm import SVC as svmgpu
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder

randomseed = 10
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

In [5]:
def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval

<IPython.core.display.Javascript object>

In [6]:
data = pd.read_csv("../dataset/seeds_dataset.txt", sep="\t", header=None)
data = shuffle(data)

le = LabelEncoder()
data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
x = data.iloc[:, :-1]
y = data.iloc[:, -1]
print(np.unique(y))

[0 1 2]


<IPython.core.display.Javascript object>

In [7]:
# data = datasets.load_iris()
# x = data.data
# y = data.target.ravel()
# print(np.unique(y))

<IPython.core.display.Javascript object>

In [8]:
xtrain, xtest, ytrain_original, ytest_original = train_test_split(
    x, y, test_size=0.3, random_state=10
)

<IPython.core.display.Javascript object>

In [9]:
xgbmodel = xgb.XGBClassifier(random_state=randomseed)
xgbmodel.fit(xtrain, ytrain_original)
print(m.accuracy_score(ytest_original, xgbmodel.predict(xtest)))

0.9365079365079365


<IPython.core.display.Javascript object>

In [10]:
cvscore = cross_val_score(xgbmodel, x, y, cv=10)
print("mean= ", np.mean(cvscore), ", std= ", np.std(cvscore))

mean=  0.9333333333333333 , std=  0.03809523809523809


<IPython.core.display.Javascript object>

Confusion matrix based models
====

In [11]:
clf_c0 = []
acc_c0 = []
predictedprobablities_c0 = []
predictedconfusionmatrix_proba_c0 = []

<IPython.core.display.Javascript object>

In [12]:
# 1. baseline classifiers

<IPython.core.display.Javascript object>

In [13]:
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 0)
# =================================================

rf_c0 = RandomForestClassifier(random_state=randomseed, n_estimators=50)
rf_c0.fit(xtrain, ytrain)
rfpred_c0 = rf_c0.predict(xtest)
print(m.accuracy_score(ytest, rfpred_c0))

clf_c0.append(rf_c0)
acc_c0.append(m.accuracy_score(ytest, rfpred_c0))
predictedprobablities_c0.append(rf_c0.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, rfpred_c0)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c0.append(propconfmat / 100)

0.9365079365079365


<IPython.core.display.Javascript object>

In [14]:
rf_c0.predict_proba(xtest)

array([[0.02, 0.98],
       [0.18, 0.82],
       [0.  , 1.  ],
       [0.06, 0.94],
       [0.22, 0.78],
       [0.  , 1.  ],
       [0.68, 0.32],
       [0.04, 0.96],
       [0.  , 1.  ],
       [0.5 , 0.5 ],
       [0.  , 1.  ],
       [0.06, 0.94],
       [0.04, 0.96],
       [1.  , 0.  ],
       [0.38, 0.62],
       [0.  , 1.  ],
       [0.64, 0.36],
       [0.  , 1.  ],
       [0.28, 0.72],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.02, 0.98],
       [0.36, 0.64],
       [0.  , 1.  ],
       [0.02, 0.98],
       [0.96, 0.04],
       [0.44, 0.56],
       [0.98, 0.02],
       [0.02, 0.98],
       [0.18, 0.82],
       [0.  , 1.  ],
       [0.06, 0.94],
       [0.06, 0.94],
       [0.72, 0.28],
       [0.9 , 0.1 ],
       [0.  , 1.  ],
       [0.26, 0.74],
       [0.52, 0.48],
       [0.  , 1.  ],
       [0.8 , 0.2 ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.82, 0.18],
       [0.66, 0.34],
       [0.  , 1.  ],
       [0.  ,

<IPython.core.display.Javascript object>

In [15]:
# rf_c0.predict_proba(xtest)
np.unique(ytest)

array([0, 1], dtype=int64)

<IPython.core.display.Javascript object>

In [18]:
p_k1_c0 = 0
p_k1_c1 = 0
for i in range(len(predictedprobablities_c0)):
    p_k1_c0 += (
        predictedprobablities_c0[i][:, 0] * predictedconfusionmatrix_proba_c0[i][0][0]
        + predictedprobablities_c0[i][:, 1] * predictedconfusionmatrix_proba_c0[i][0][1]
    )

    p_k1_c1 += (
        predictedprobablities_c0[0][:, 0] * predictedconfusionmatrix_proba_c0[0][1][0]
        + predictedprobablities_c0[0][:, 1] * predictedconfusionmatrix_proba_c0[0][1][1]
    )

    print(i)
p_k1_c0 = p_k1_c0 / len(predictedprobablities_c0)
p_k1_c1 = p_k1_c1 / len(predictedprobablities_c0)

0


<IPython.core.display.Javascript object>

In [19]:
finalcol = np.zeros((ytest.shape[0], 2))
finalcol[:, 0] = p_k1_c0
finalcol[:, 1] = p_k1_c1
finalpred = np.argmax(finalcol, axis=1)
ytrain, ytest = swapcolumns(ytrain, ytest, 0)

print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

0.9365079365079365
[[18  4]
 [ 0 41]]


<IPython.core.display.Javascript object>

In [20]:
# ytrain = ytrain_original.copy()
# ytest = ytest_original.copy()
# ytrain, ytest = swapcolumns(ytrain, ytest, 0)
# # =================================================
# xgbmodel_c0 = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
# xgbmodel_c0.fit(xtrain, ytrain)
# xgbmodelpred_c0 = xgbmodel_c0.predict(xtest)
# print(m.accuracy_score(ytest, xgbmodelpred_c0))

# clf_c0.append(xgbmodel_c0)
# acc_c0.append(m.accuracy_score(ytest, xgbmodelpred_c0))
# predictedprobablities_c0.append(xgbmodel_c0.predict_proba(xtest))

# confmat = m.confusion_matrix(ytest, xgbmodelpred_c0)
# confsumh = np.sum(confmat, axis=0)
# propconfmat = confmat.copy()
# for i in range(propconfmat.shape[0]):
#     propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
# predictedconfusionmatrix_proba_c0.append(propconfmat / 100)


<IPython.core.display.Javascript object>

In [21]:
clf_c1 = []
acc_c1 = []
predictedprobablities_c1 = []
predictedconfusionmatrix_proba_c1 = []

# =================================================
# classs 1
# =================================================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 1)
# =================================================

rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
rf.fit(xtrain, ytrain)
rfpred = rf.predict(xtest)
print(m.accuracy_score(ytest, rfpred))

clf_c1.append(rf)
acc_c1.append(m.accuracy_score(ytest, rfpred))
predictedprobablities_c1.append(rf.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c1.append(propconfmat / 100)


0.9682539682539683


<IPython.core.display.Javascript object>

In [22]:
print(rf.predict_proba(xtest))

[[0.   1.  ]
 [0.74 0.26]
 [0.98 0.02]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.   1.  ]
 [0.98 0.02]
 [1.   0.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.92 0.08]
 [0.92 0.08]
 [0.   1.  ]
 [0.68 0.32]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.42 0.58]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.4  0.6 ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.98 0.02]
 [0.   1.  ]
 [0.36 0.64]
 [0.06 0.94]
 [1.   0.  ]
 [0.   1.  ]
 [0.12 0.88]
 [1.   0.  ]
 [0.14 0.86]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.42 0.58]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.98 0.02]
 [0.4  0.6 ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.92 0.08]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.74 0.26]
 [1.   0.  ]]


<IPython.core.display.Javascript object>

In [24]:
p_k2_c0 = 0
p_k2_c1 = 0
for i in range(len(predictedprobablities_c1)):
    p_k2_c0 += (
        predictedprobablities_c1[i][:, 0] * predictedconfusionmatrix_proba_c1[i][0][0]
        + predictedprobablities_c1[i][:, 1] * predictedconfusionmatrix_proba_c1[i][0][1]
    )

    p_k2_c1 += (
        predictedprobablities_c1[i][:, 0] * predictedconfusionmatrix_proba_c1[i][1][0]
        + predictedprobablities_c1[i][:, 1] * predictedconfusionmatrix_proba_c1[i][1][1]
    )

    print(i)
p_k2_c0 = p_k2_c0 / len(predictedprobablities_c0)
p_k2_c1 = p_k2_c1 / len(predictedprobablities_c0)

0


<IPython.core.display.Javascript object>

In [25]:
finalcol = np.zeros((ytest.shape[0], 2))
finalcol[:, 0] = p_k2_c0
finalcol[:, 1] = p_k2_c1
finalpred = np.argmax(finalcol, axis=1)
ytrain, ytest = swapcolumns(ytrain, ytest, 0)

print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

0.9682539682539683
[[20  2]
 [ 0 41]]


<IPython.core.display.Javascript object>

In [26]:
# ytrain = ytrain_original.copy()
# ytest = ytest_original.copy()
# ytrain, ytest = swapcolumns(ytrain, ytest, 1)
# # =================================================
# xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
# xgbmodel.fit(xtrain, ytrain)
# xgbmodelpred = xgbmodel.predict(xtest)
# print(m.accuracy_score(ytest, xgbmodelpred))

# clf_c1.append(xgbmodel)
# acc_c1.append(m.accuracy_score(ytest, xgbmodelpred))
# predictedprobablities_c1.append(xgbmodel.predict_proba(xtest))

# confmat = m.confusion_matrix(ytest, xgbmodelpred)
# confsumh = np.sum(confmat, axis=0)
# propconfmat = confmat.copy()
# for i in range(propconfmat.shape[0]):
#     propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
# predictedconfusionmatrix_proba_c1.append(propconfmat / 100)


<IPython.core.display.Javascript object>

In [27]:
#=================================================
# classs 2
#=================================================

clf_c2 = []
acc_c2 = []
predictedprobablities_c2 = []
predictedconfusionmatrix_proba_c2 = []

ytrain=ytrain_original.copy()
ytest=ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,2)

#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=50)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.accuracy_score(ytest,rfpred))

clf_c2.append(rf)
acc_c2.append(m.accuracy_score(ytest,rfpred))
predictedprobablities_c2.append(rf.predict_proba(xtest))

confmat=m.confusion_matrix(ytest,rfpred)
confsumh=np.sum(confmat,axis=0)
propconfmat=confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
predictedconfusionmatrix_proba_c2.append(propconfmat/100)


0.9047619047619048


<IPython.core.display.Javascript object>

In [28]:
# ytrain = ytrain_original.copy()
# ytest = ytest_original.copy()
# ytrain, ytest = swapcolumns(ytrain, ytest, 2)
# # =================================================
# xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
# xgbmodel.fit(xtrain, ytrain)
# xgbmodelpred = xgbmodel.predict(xtest)
# print(m.accuracy_score(ytest, xgbmodelpred))

# clf_c2.append(xgbmodel)
# acc_c2.append(m.accuracy_score(ytest, xgbmodelpred))
# predictedprobablities_c2.append(xgbmodel.predict_proba(xtest))

# confmat = m.confusion_matrix(ytest, xgbmodelpred)
# confsumh = np.sum(confmat, axis=0)
# propconfmat = confmat.copy()
# for i in range(propconfmat.shape[0]):
#     propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
# predictedconfusionmatrix_proba_c2.append(propconfmat / 100)


<IPython.core.display.Javascript object>

In [29]:
predictedprobablities_c2

[array([[1.  , 0.  ],
        [0.02, 0.98],
        [0.  , 1.  ],
        [0.94, 0.06],
        [0.76, 0.24],
        [0.  , 1.  ],
        [0.56, 0.44],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [0.38, 0.62],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [1.  , 0.  ],
        [0.58, 0.42],
        [1.  , 0.  ],
        [0.78, 0.22],
        [0.  , 1.  ],
        [1.  , 0.  ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [1.  , 0.  ],
        [1.  , 0.  ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [1.  , 0.  ],
        [0.8 , 0.2 ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [1.  , 0.  ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [0.82, 0.18],
        [0.02, 0.98],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [1.  , 0.  ],
        [1.  , 0.  ],
        [1.  , 0.  ],
        [0.  , 1.  ],
        [0.  , 1.  ],
        [0

<IPython.core.display.Javascript object>

In [30]:
print((rf.predict_proba(xtest)))

[[1.   0.  ]
 [0.02 0.98]
 [0.   1.  ]
 [0.94 0.06]
 [0.76 0.24]
 [0.   1.  ]
 [0.56 0.44]
 [0.   1.  ]
 [0.   1.  ]
 [0.38 0.62]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.58 0.42]
 [1.   0.  ]
 [0.78 0.22]
 [0.   1.  ]
 [1.   0.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [1.   0.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.8  0.2 ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.82 0.18]
 [0.02 0.98]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.04 0.96]
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]
 [0.   1.  ]
 [0.   1.  ]
 [0.96 0.04]
 [0.08 0.92]
 [0.04 0.96]
 [0.   1.  ]
 [0.   1.  ]
 [0.06 0.94]
 [0.36 0.64]
 [1.   0.  ]
 [1.   0.  ]
 [0.   1.  ]
 [0.   1.  ]]


<IPython.core.display.Javascript object>

In [31]:
p_k3_c0 = 0
p_k3_c1 = 0
for i in range(len(predictedprobablities_c2)):
    p_k3_c0 += (
        predictedprobablities_c2[i][:, 0] * predictedconfusionmatrix_proba_c2[i][0][0]
        + predictedprobablities_c2[i][:, 1] * predictedconfusionmatrix_proba_c2[i][0][1]
    )

    p_k3_c1 += (
        predictedprobablities_c2[i][:, 0] * predictedconfusionmatrix_proba_c2[i][1][0]
        + predictedprobablities_c2[i][:, 1] * predictedconfusionmatrix_proba_c2[i][1][1]
    )

    print(i)
p_k3_c0 = p_k3_c0 / len(predictedprobablities_c2)
p_k3_c1 = p_k3_c1 / len(predictedprobablities_c2)

0


<IPython.core.display.Javascript object>

In [32]:
finalcol = np.zeros((ytest.shape[0], 2))
finalcol[:, 0] = p_k3_c0
finalcol[:, 1] = p_k3_c1
finalpred = np.argmax(finalcol, axis=1)
ytrain, ytest = swapcolumns(ytrain, ytest, 0)

print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

0.9365079365079365
[[19  0]
 [ 4 40]]


<IPython.core.display.Javascript object>

In [33]:
# # pc1_all = []
# # pc2_all = []

# # for i in range(len(predictedconfusionmatrix_proba)):

# #     pc1 = ( predictedprobablities[i][:, 0]    )
# #     pc2 = (  predictedprobablities[i][:, 1]    )

# #     pc1_all.append(pc1)
# #     pc2_all.append(pc2)



# ci0 = ci1 = ci2 = 1 

# p_k1_c1 = ci0 * (
#     predictedprobablities[0][:, 0] * predictedconfusionmatrix_proba[0][1][0]
#     + predictedprobablities[0][:, 1] * predictedconfusionmatrix_proba[0][1][1]
# )


# p_k2_c1 = ci0 * (
#     predictedprobablities[1][:, 0] * predictedconfusionmatrix_proba[1][1][0]
#     + predictedprobablities[1][:, 1] * predictedconfusionmatrix_proba[1][1][1]
# )

# p_k3_c1 = ci0 * (
#     predictedprobablities[2][:, 0] * predictedconfusionmatrix_proba[2][1][0]
#     + predictedprobablities[2][:, 1] * predictedconfusionmatrix_proba[2][1][1]
# )


# p_k1_c0 = ci0 * (
#     predictedprobablities[0][:, 0] * predictedconfusionmatrix_proba[0][0][0]
#     + predictedprobablities[0][:, 1] * predictedconfusionmatrix_proba[0][0][1]
# ) + (p_k2_c1 * p_k3_c1)


# p_k2_c0 = ci0 * (
#     predictedprobablities[1][:, 0] * predictedconfusionmatrix_proba[1][0][0]
#     + predictedprobablities[1][:, 1] * predictedconfusionmatrix_proba[1][0][1]
# ) + (p_k1_c1 * p_k3_c1)

# p_k3_c0 = ci0 * (
#     predictedprobablities[2][:, 0] * predictedconfusionmatrix_proba[2][0][0]
#     + predictedprobablities[2][:, 1] * predictedconfusionmatrix_proba[2][0][1]
# ) + (p_k2_c1 * p_k1_c1)

# finalcol = np.zeros((ytest.shape[0], 3))
# finalcol[:, 0] = p_k1_c0
# finalcol[:, 1] = p_k2_c0
# finalcol[:, 2] = p_k3_c0
# finalpred = np.argmax(finalcol, axis=1)

# ytest = ytest_original.copy()
# print(m.accuracy_score(ytest, finalpred))
# print(m.confusion_matrix(ytest, finalpred))


<IPython.core.display.Javascript object>

In [36]:
p_k1_c0 = 0
p_k1_c1 = 0
for i in range(len(predictedprobablities_c0)):
    p_k1_c0 += (
        predictedprobablities_c0[i][:, 0] * predictedconfusionmatrix_proba_c0[i][0][0]
        + predictedprobablities_c0[i][:, 1] * predictedconfusionmatrix_proba_c0[i][0][1]
    )

    p_k1_c1 += (
        predictedprobablities_c0[0][:, 0] * predictedconfusionmatrix_proba_c0[0][1][0]
        + predictedprobablities_c0[0][:, 1] * predictedconfusionmatrix_proba_c0[0][1][1]
    )

    print(i)
p_k1_c0 = p_k1_c0 / len(predictedprobablities_c0)
p_k1_c1 = p_k1_c1 / len(predictedprobablities_c0)

0


<IPython.core.display.Javascript object>

In [35]:
ytrain, ytest = swapcolumns(ytrain, ytest, 0)

finalcol = np.zeros((ytest.shape[0], 2))
finalcol[:, 0] = p_k1_c0
finalcol[:, 1] = p_k1_c1
finalpred = np.argmax(finalcol, axis=1)
ytrain, ytest = swapcolumns(ytrain, ytest, 0)

print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

0.4126984126984127
[[ 0 19]
 [18 26]]


<IPython.core.display.Javascript object>

In [ ]:
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 0)


finalcol = np.zeros((ytest.shape[0], 2))
finalcol[:, 0] = p_k1_c0
finalcol[:, 1] = p_k1_c1
finalpred = np.argmax(predictedprobablities_c0[0], axis=1)

ytest = ytest_original.copy()
print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

In [30]:
p_k2_c0 = 0
p_k2_c1 = 0
for i in range(len(predictedprobablities_c1)):
    p_k2_c0 += (
        predictedprobablities_c1[i][:, 0] * predictedconfusionmatrix_proba_c1[i][0][0]
        + predictedprobablities_c1[i][:, 1] * predictedconfusionmatrix_proba_c1[i][0][1]
    )

    p_k2_c1 += (
        predictedprobablities_c1[i][:, 0] * predictedconfusionmatrix_proba_c1[i][1][0]
        + predictedprobablities_c1[i][:, 1] * predictedconfusionmatrix_proba_c1[i][1][1]
    )

    print(i)
p_k2_c0 = p_k2_c0 / len(predictedprobablities_c0)
p_k2_c1 = p_k2_c1 / len(predictedprobablities_c0)

0


<IPython.core.display.Javascript object>

In [31]:
ytrain, ytest = swapcolumns(ytrain, ytest, 1)

finalcol = np.zeros((ytest.shape[0], 2))
finalcol[:, 0] = p_k2_c0
finalcol[:, 1] = p_k2_c1
finalpred = np.argmax(finalcol, axis=1)
ytrain, ytest = swapcolumns(ytrain, ytest, 0)

print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

0.3333333333333333
[[ 0 22]
 [20 21]]


<IPython.core.display.Javascript object>

In [41]:
p_k3_c0 = 0
p_k3_c1 = 0
for i in range(len(predictedprobablities_c2)):
    p_k3_c0 += (
        predictedprobablities_c2[i][:, 0] * predictedconfusionmatrix_proba_c2[i][0][0]
        + predictedprobablities_c2[i][:, 1] * predictedconfusionmatrix_proba_c2[i][0][1]
    )

    p_k3_c1 += (
        predictedprobablities_c2[i][:, 0] * predictedconfusionmatrix_proba_c2[i][1][0]
        + predictedprobablities_c2[i][:, 1] * predictedconfusionmatrix_proba_c2[i][1][1]
    )

    print(i)
p_k3_c0 = p_k3_c0 / len(predictedprobablities_c2)
p_k3_c1 = p_k3_c1 / len(predictedprobablities_c2)

0


<IPython.core.display.Javascript object>

In [42]:
ytrain, ytest = swapcolumns(ytrain, ytest, 2)

finalcol = np.zeros((ytest.shape[0], 2))
finalcol[:, 0] = p_k3_c0
finalcol[:, 1] = p_k3_c1
finalpred = np.argmax(finalcol, axis=1)
ytrain, ytest = swapcolumns(ytrain, ytest, 0)

print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

0.6349206349206349
[[ 0  0]
 [23 40]]


<IPython.core.display.Javascript object>

In [24]:

# ci0 = ci1 = ci2 = 1 
 


p_k1_c0 = p_k1_c0 #+ (p_k2_c1 * p_k3_c1)


p_k2_c0 =p_k2_c0 #+ (p_k1_c1 * p_k3_c1)

p_k3_c0 = p_k3_c0 #+ (p_k2_c1 * p_k1_c1)



<IPython.core.display.Javascript object>

In [25]:
finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = p_k1_c0
finalcol[:, 1] = p_k2_c0
finalcol[:, 2] = p_k3_c0
finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

0.7936507936507936
[[15  0  7]
 [ 0 16  6]
 [ 0  0 19]]


<IPython.core.display.Javascript object>

In [23]:

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = p_k1_c1
finalcol[:, 1] = p_k2_c1
finalcol[:, 2] = p_k3_c1
finalpred = np.argmin(finalcol, axis=1)

ytest = ytest_original.copy()
print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))


0.36507936507936506
[[ 4  0 18]
 [ 0  0 22]
 [ 0  0 19]]


<IPython.core.display.Javascript object>